In [1]:
def max_pagination_pages(soup):
    page_items = soup.find_all('li', class_='page-item')
    page_values = [item.get_text(strip=True) for item in page_items]

    pageList = [int(value) for value in page_values if value.isdigit()]
    if pageList:
        max_page = max(pageList)
        return max_page
    else:
        return None
        
def pagesoup_to_company_data(soup):
    companies_data = []

    company_blocks = soup.find_all('div', class_='row data-row')

    for block in company_blocks:
        name_tag = block.find('a')
        if name_tag:
            company_name = name_tag.get_text(strip=True)
            url = name_tag['href'] if name_tag['href'].endswith("brands") else f"{name_tag['href']}/brands"
            
            details_tag = block.find_all('div', class_='col-xs-12')[-1]
            # print(details_tag)
            if details_tag:
                details_text = details_tag.get_text(strip=True).split(',')
                generics_str = details_text[0].replace("generics", "").strip()
                brand_names_str = details_text[1].replace("brand names", "").strip()

                generics = int(generics_str) if generics_str.isdigit() else 0
                brand_names = int(brand_names_str) if generics_str.isdigit() else 0
            
                companies_data.append({
                    'name': company_name,
                    'generics': generics,
                    'brand_names': brand_names,
                    'company_url': url
                })
    return companies_data


def pagesoup_to_brand_data(soup, company_name = ""):
    brand_blocks = soup.find_all('a', class_='hoverable-block')

    brands_data = []
    for brand_block in brand_blocks:
        print(brand_block)
        print("--------")
        break
        name_elem = brand_block.find('div', class_='data-row-top')
        dosage_elem = brand_block.find('span', class_='inline-dosage-form')
        strength_elem = brand_block.find('span', class_='grey-ligten')
        generic_elem = brand_block.find_all('div')[3]  # Assuming the generic name is in the third div
        price_elem = brand_block.find('span', class_='package-pricing')

        url = brand_block['href'] if 'href' in brand_block.attrs else ''
        dosage = dosage_elem.get_text(strip=True) if dosage_elem else ''
        name = (name_elem.get_text(strip=True) if name_elem else '').strip(dosage)  # Exclude dosage form
        strength = strength_elem.get_text(strip=True) if strength_elem else ''
        generic = generic_elem.get_text(strip=True) if generic_elem else ''
        price = price_elem.get_text(strip=True).replace('Unit Price : ', '') if price_elem else ''
        
        brands_data.append({
            "company": company_name,
            'url': url,
            'name': name,
            'dosage': dosage,
            'strength': strength,
            'generic': generic,
            'price': price
        })
    return brands_data



def pagesoup_to_medicine_info(soup):
    for br in soup.find_all('br'):
        br.replace_with('\n')

    medication_info = []

    indications_divs = soup.find_all('div', class_='ac-body')

    for indications_div in indications_divs:
        if indications_div.parent:
            parent_div = indications_div.parent
            if len(parent_div.find_all('div')) == 2:  # Check if it has exactly two child divs
                child_divs = parent_div.find_all('div')
                medication_info.append({
                    "section": child_divs[0].get_text(strip=True),
                    "text": child_divs[1].get_text(strip=True),
                })
    
    return medication_info

In [1]:
import pandas as pd

def convert_parquet_to_csv(parquet_file, csv_file):
    # Read the parquet file
    df = pd.read_parquet(parquet_file)
    # Convert to CSV
    df.to_csv(csv_file, index=False)

# Example usage
convert_parquet_to_csv('train-00000-of-00001.parquet', 'medex_db.csv')


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [8]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 546.6 kB/s eta 0:00:0000:0100:02


In [3]:
## All Companies Page to Company Data

import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint

base_url = "http://medex.com.bd/companies?page=1"

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

max_company_pages = max_pagination_pages(soup)
max_company_pages

all_company_info = []
print(f"max_company_pages: {max_company_pages}")
for page in range(max_company_pages):
    base_url = "http://medex.com.bd/companies"

    response = requests.get(
        base_url,
        params={
            'page': page + 1
            }
        )
    # pprint(response.json())
    soup = BeautifulSoup(response.content, 'html.parser')
    company_data = pagesoup_to_company_data(soup)
    all_company_info.extend(company_data)
    pprint(len(company_data))


print("Total Companies: ", len(all_company_info))

max_company_pages: 7
30
30
30
30
30
30
22
Total Companies:  202


In [ ]:
all_company_info

[{'name': 'ACI Limited',
  'generics': 380,
  'brand_names': 731,
  'company_url': 'https://medex.com.bd/companies/2/aci-limited/brands'},
 {'name': 'ACME Laboratories Ltd.',
  'generics': 434,
  'brand_names': 739,
  'company_url': 'https://medex.com.bd/companies/3/acme-laboratories-ltd/brands'},
 {'name': 'Ad-din Pharmaceuticals Ltd.',
  'generics': 75,
  'brand_names': 108,
  'company_url': 'https://medex.com.bd/companies/103/ad-din-pharmaceuticals-ltd/brands'},
 {'name': 'Aexim Pharmaceuticals Ltd.',
  'generics': 34,
  'brand_names': 43,
  'company_url': 'https://medex.com.bd/companies/114/aexim-pharmaceuticals-ltd/brands'},
 {'name': 'Al-Madina Pharmaceuticals Ltd.',
  'generics': 51,
  'brand_names': 79,
  'company_url': 'https://medex.com.bd/companies/292/al-madina-pharmaceuticals-ltd/brands'},
 {'name': 'Albion Laboratories Limited',
  'generics': 115,
  'brand_names': 172,
  'company_url': 'https://medex.com.bd/companies/108/albion-laboratories-limited/brands'},
 {'name': 'Al

In [23]:
all_company_info

[]

In [24]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint


company_name = "aci-limited"  #input("Enter the company name (use hyphens for spaces): ")
company_url = "https://medex.com.bd/companies/2/aci-limited/brands" 


response = requests.get(company_url)
# pprint(response.json())
soup = BeautifulSoup(response.content, 'html.parser')

max_pages = max_pagination_pages(soup)
max_pages

all_brands_info = []
for page in range(max_pages):
    base_url = company_url

    response = requests.get(
        base_url,
        params={
            'page': page + 1
            }
        )
    # pprint(response.json())
    soup = BeautifulSoup(response.content, 'html.parser')
    brand_data = pagesoup_to_brand_data(soup)
    all_brands_info.extend(brand_data)
    pprint(len(brand_data))


print("Total Brands: ", len(all_brands_info))

TypeError: 'NoneType' object cannot be interpreted as an integer

In [6]:
all_brands_info

[{'company': '',
  'url': 'https://medex.com.bd/brands/30731/abaclor-250-mg-capsule',
  'name': 'Abaclor',
  'dosage': 'Capsule',
  'strength': '250 mg',
  'price': '৳ 21.14'},
 {'company': '',
  'url': 'https://medex.com.bd/brands/7097/abaclor-500-mg-capsule',
  'name': 'Abaclor',
  'dosage': 'Capsule',
  'strength': '500 mg',
  'price': '৳ 40.27'},
 {'company': '',
  'url': 'https://medex.com.bd/brands/7098/abaclor-125-mg-suspension',
  'name': 'Abaclor',
  'dosage': 'Suspension',
  'strength': '125 mg/5 ml',
  'price': '৳ 280.00'},
 {'company': '',
  'url': 'https://medex.com.bd/brands/7099/abaclor-125-mg-pediatric-drop',
  'name': 'Abacl',
  'dosage': 'Pediatric Drop',
  'strength': '125 mg/1.25 ml',
  'price': '৳ 135.00'},
 {'company': '',
  'url': 'https://medex.com.bd/brands/3075/abecab-5-mg-tablet',
  'name': 'Abec',
  'dosage': 'Tablet',
  'strength': '5 mg+20 mg',
  'price': '৳ 12.00'},
 {'company': '',
  'url': 'https://medex.com.bd/brands/3076/abecab-5-mg-tablet',
  'name':

In [8]:
all_brands_info[-1]

{'company': '',
 'url': 'https://medex.com.bd/brands/35834/zoldix-5-mg-injection',
 'name': 'Zoldix',
 'dosage': 'Injection',
 'strength': '5 mg/100 ml',
 'price': '৳ 6,000.00'}

In [26]:
from pprint import pprint

medicine_url = "https://medex.com.bd/brands/35834/zoldix-5-mg-injection"


response = requests.get(medicine_url)
soup = BeautifulSoup(response.text, 'html.parser')
en_medication_info = pagesoup_to_medicine_info(soup)


response = requests.get(f"{medicine_url}/bn")
soup = BeautifulSoup(response.text, 'html.parser')
bn_medication_info = pagesoup_to_medicine_info(soup)

medication_info = {
    "bn": bn_medication_info,
    "en": en_medication_info
}


pprint(medication_info)





{'bn': [{'section': 'নির্দেশনা',
         'text': 'যোলেড্রোনিক এসিড নিম্নোক্ত উপসর্গে নির্দেশিত-পোস্টম্যানোপসাল '
                 'অস্টিওপোরোসিসের চিকিৎসা এবং প্রতিরোধের জন্যঅস্টিওপোরোসিসে '
                 'আক্রান্ত পুরুষদের হাড়ের ভর বাড়ানোর জন্যগ্লুকোকর্টিকয়েড '
                 'প্ররোচিত অস্টিওপোরোসিসের চিকিৎসা এবং প্রতিরোধের জন্যপুরুষ ও '
                 'মহিলাদের হাড়ের পেজেট রোগের চিকিৎসার জন্যব্যবহারের '
                 'সীমাবদ্ধতা: ব্যবহারের সর্বোত্তম সময়কাল নির্ধারণ করা হয়নি। '
                 'ফ্র্যাকচারের কম ঝুঁকিতে থাকা রোগীদের জন্য, ৩ থেকে ৫ বছর '
                 'ব্যবহারের পরে ওষুধ বন্ধ করার কথা বিবেচনা করতে হবে।'},
        {'section': 'ফার্মাকোলজি',
         'text': 'যোলেড্রোনিক এসিড একটি বিসফসফোনেট যেটি মূলত হাড়ের উপর কাজ '
                 'করে। এটি অস্টিওক্লাস্ট মিডিয়েটেড হাড়ের রিসর্পশন বাধা দেয়। '
                 'বিসফসফোনেটর হাড়ে নির্দিষ্ট ভাবে কাজ করা নির্ভর করে '
                 'মিনারেলাইজড হাড়ের প্রতি এটির অধিক আকর্ষণের উপর। '
                

In [24]:
from pprint import pprint

response = requests.get("https://medex.com.bd/brands/35834/zoldix-5-mg-injection/bn")
soup = BeautifulSoup(response.text, 'html.parser')

for br in soup.find_all('br'):
    br.replace_with('\n')

medication_info = []

indications_divs = soup.find_all('div', class_='ac-body')

for indications_div in indications_divs:
    if indications_div.parent:
        parent_div = indications_div.parent
        if len(parent_div.find_all('div')) == 2:  # Check if it has exactly two child divs
            child_divs = parent_div.find_all('div')
            medication_info.append({
                "section": child_divs[0].get_text(strip=True),
                "text": child_divs[1].get_text(strip=True),
            })
pprint(medication_info)







[{'section': 'নির্দেশনা',
  'text': 'যোলেড্রোনিক এসিড নিম্নোক্ত উপসর্গে নির্দেশিত-পোস্টম্যানোপসাল '
          'অস্টিওপোরোসিসের চিকিৎসা এবং প্রতিরোধের জন্যঅস্টিওপোরোসিসে আক্রান্ত '
          'পুরুষদের হাড়ের ভর বাড়ানোর জন্যগ্লুকোকর্টিকয়েড প্ররোচিত '
          'অস্টিওপোরোসিসের চিকিৎসা এবং প্রতিরোধের জন্যপুরুষ ও মহিলাদের হাড়ের '
          'পেজেট রোগের চিকিৎসার জন্যব্যবহারের সীমাবদ্ধতা: ব্যবহারের সর্বোত্তম '
          'সময়কাল নির্ধারণ করা হয়নি। ফ্র্যাকচারের কম ঝুঁকিতে থাকা রোগীদের '
          'জন্য, ৩ থেকে ৫ বছর ব্যবহারের পরে ওষুধ বন্ধ করার কথা বিবেচনা করতে '
          'হবে।'},
 {'section': 'ফার্মাকোলজি',
  'text': 'যোলেড্রোনিক এসিড একটি বিসফসফোনেট যেটি মূলত হাড়ের উপর কাজ করে। এটি '
          'অস্টিওক্লাস্ট মিডিয়েটেড হাড়ের রিসর্পশন বাধা দেয়। বিসফসফোনেটর '
          'হাড়ে নির্দিষ্ট ভাবে কাজ করা নির্ভর করে মিনারেলাইজড হাড়ের প্রতি '
          'এটির অধিক আকর্ষণের উপর। ইন্ট্রাভেনাস ইনজেকশনের মাধ্যমে প্রয়োগ করা '
          'যোলেড্রোনিক এসিড হাড়ের দ্রুত প্রবেশ করে এবং অস্থির উচ্চ '
 

In [35]:
## All Companies Page to Company Data

import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint
from tqdm.auto import tqdm


for company_data in tqdm(all_company_info):
    company_name = company_data["name"]
    company_url = company_data["company_url"]
    response = requests.get(company_url)
    # pprint(response.json())

    soup = BeautifulSoup(response.content, 'html.parser')

    max_pages = max_pagination_pages(soup)
    # max_pages

    all_brands_info = []
    for page in tqdm(range(max_pages)):
        base_url = company_url

        response = requests.get(
            base_url,
            params={
                'page': page + 1
                }
            )
        # pprint(response.json())
        soup = BeautifulSoup(response.content, 'html.parser')
        brands_data = pagesoup_to_brand_data(soup, company_name=company_name)

        for brand_data in tqdm(brands_data, leave=False):
            medicine_url = brand_data["url"]

            response = requests.get(medicine_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            en_medication_info = pagesoup_to_medicine_info(soup)


            response = requests.get(f"{medicine_url}/bn")
            soup = BeautifulSoup(response.text, 'html.parser')
            bn_medication_info = pagesoup_to_medicine_info(soup)

            medication_info = {
                "bn": bn_medication_info,
                "en": en_medication_info
            }
            brand_data["info"] = medication_info
            
            all_brands_info.append(brand_data)
    pprint(f"{company_name}: {len(all_brands_info)}")


    company_data["all_brands"] = brand_data
    all_company_info.extend(company_data)

# pprint(len(company_data))


# print("Total Companies: ", len(all_company_info))

  0%|          | 0/201 [00:00<?, ?it/s]


fff


fff


fff


fff


fff


fff


fff



  0%|          | 0/201 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import csv

data = [
    ['নাম', 'বয়স', 'ঠিকানা'],
    ['রাহিম', '২৫', 'ঢাকা'],
    ['করিম', '৩০', 'চট্টগ্রাম']
]

with open('bangla_data.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)

### Scrape generic Info

In [9]:
def pagesoup_to_generic(soup):
    generics = []
    
    # Find all generic name links
    for item in soup.select('.hoverable-block'):
        title = item.select_one('.dcind-title').get_text(strip=True)
        brands_text = item.select_one('.dcind').get_text(strip=True)
        ncd_count = int(brands_text.split()[0])  # Extract the number of available brands
        url = item['href']
        
        generics.append({
            'generic_name': title,
            'brand_count': ncd_count,
            'url': url
        })
    
    return generics

In [10]:
# Allopathic


## All Companies Page to Company Data

import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint
from tqdm.auto import tqdm

base_url = "https://medex.com.bd/generics"

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

max_generic_pages = max_pagination_pages(soup)
max_generic_pages


all_generic_info = []
print(f"max_generic_pages: {max_generic_pages}")
for page in tqdm(range(max_company_pages)):
    base_url = "https://medex.com.bd/generics"

    response = requests.get(
        base_url,
        params={
            'page': page + 1
            }
        )
    # pprint(response.json())
    soup = BeautifulSoup(response.content, 'html.parser')
    generics = pagesoup_to_generic(soup)
    all_generic_info.extend(generics)


max_generic_pages: 82


100%|██████████| 82/82 [00:33<00:00,  2.47it/s]


In [12]:
len(all_generic_info)

1629

In [15]:
# Allopathic


## All Companies Page to Company Data

import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint
from tqdm.auto import tqdm

base_url = "https://medex.com.bd/generics?herbal=1"

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

max_generic_pages = max_pagination_pages(soup)
print("All Herbel Generic: {max_generic_pages}")


all_generic_info = []
print(f"max_generic_pages: {max_generic_pages}")
for page in tqdm(range(max_company_pages)):
    base_url = "https://medex.com.bd/generics?herbal=1"

    response = requests.get(
        base_url,
        params={
            'page': page + 1
            }
        )
    # pprint(response.json())
    soup = BeautifulSoup(response.content, 'html.parser')
    generics = pagesoup_to_generic(soup)
    all_generic_info.extend(generics)


All Herbel Generic: {max_generic_pages}
max_generic_pages: 15


100%|██████████| 82/82 [00:28<00:00,  2.90it/s]


In [17]:
len(all_generic_info)
all_generic_info

[{'generic_name': '23 herb formula to increase breast milk',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/1833/23-herb-formula-to-increase-breast-milk'},
 {'generic_name': '5-Hydroxytryptophan + Valerian extract',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/2163/5-hydroxytryptophan-valerian-extract'},
 {'generic_name': '6s-5-Methyltetrahydrofolate',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/2304/6s-5-methyltetrahydrofolate'},
 {'generic_name': 'Aab-e Hayat',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/2268/aab-e-hayat'},
 {'generic_name': 'Abhayarista',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/1752/abhayarista'},
 {'generic_name': 'Acacia + Dates + Asparagus + Edible Pine',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/1715/acacia-dates-asparagus-edible-pine'},
 {'generic_name': 'Achillea millefolium + Artemisia absinthium',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/2235/

In [18]:
import json 

with open('all_herbel_generics.json', 'w', encoding='utf-8') as f:
    json.dump(all_generic_info, f, indent=4)


In [20]:

import requests
from bs4 import BeautifulSoup
import csv
import re
from pprint import pprint
from tqdm.auto import tqdm

base_url = "https://medex.com.bd/generics/2268/aab-e-hayat"

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

pagesoup_to_medicine_info(soup)

[{'section': 'Indications',
  'text': 'Headache, Toothache, Catarrh, Diarrhea, vomiting, gastritis, Indigestion, Anorexia, flatulence.'},
 {'section': 'Composition',
  'text': 'Each 5 ml contains following Ingredients (as aqua extract):Cinnamomum camphora: 1.615 gmTrachyspermum ammi: 0.807 gmMentha piperita: 0.404 gmOil-Eucalyptus: 0.519 mlPinus longifolia: 0.259 mlCarum carvi: 0.259 mlPimpinella anisum: 0.192 mlZingiber officinale: 0.192 ml'},
 {'section': 'Pharmacology',
  'text': 'Hayat is an effective natural medicine for headache & Common Cold. Accordi BNUF Hayat Solution contains valuable ingredients.Cinnamomum camphora: The gum resin is antiseptic, analgesic, antispasmodic, anti-rheumatic. By producing a local anesthetic action it can relief pain. It is used in medicine internally for its calming influence in hysteria, nervousness and neuralgia. It is also used in serious diarrhea and for clearing mucous from the upper respiratory tract. Camphor is most commonly used externally 

In [34]:
with open('all_alopathic_generics.json', 'r', encoding='utf-8') as f:
    all_alopathic_generics = json.load(f)

len(all_alopathic_generics)

1629

In [49]:
from tqdm import tqdm

with open('all_alopathic_generics_details.json', 'r', encoding='utf-8') as f:
    all_alopathic_generics = json.load(f)

len(all_alopathic_generics)

for n, generic_info in tqdm(enumerate(all_alopathic_generics)):
    if n>=528:
        continue
    url = generic_info["url"]

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    en_info = pagesoup_to_medicine_info(soup)

    bn_url = url + "/bn"
    response = requests.get(bn_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    bn_info = pagesoup_to_medicine_info(soup)

    lang_info = {
        "bn": bn_info,
        "en": en_info,
    }
    all_alopathic_generics[n]["info"] = lang_info

len(all_alopathic_generics)

# Efavirenz

1629it [10:30,  2.58it/s]


1629

In [50]:
import json 
with open('all_alopathic_generics_details.json', 'w', encoding='utf-8') as f:
    json.dump(all_alopathic_generics, f, indent=4, ensure_ascii=False)

    

In [44]:
sum([1 if "info" in generic_info else 0 for generic_info in all_alopathic_generics])

530

In [42]:
from tqdm import tqdm

with open('all_herbel_generics.json', 'r', encoding='utf-8') as f:
    all_herbal_generics = json.load(f)

len(all_herbal_generics)

for n, generic_info in tqdm(enumerate(all_herbal_generics)):
    url = generic_info["url"]

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    en_info = pagesoup_to_medicine_info(soup)

    bn_url = url + "/bn"
    response = requests.get(bn_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    bn_info = pagesoup_to_medicine_info(soup)

    lang_info = {
        "bn": bn_info,
        "en": en_info,
    }
    all_herbal_generics[n]["info"] = lang_info

    # break

10it [00:07,  1.38it/s]


KeyboardInterrupt: 

In [30]:
all_herbal_generics

[{'generic_name': '23 herb formula to increase breast milk',
  'brand_count': 1,
  'url': 'https://medex.com.bd/generics/1833/23-herb-formula-to-increase-breast-milk',
  'info': {'bn': [{'section': 'Indications',
     'text': 'This is a natural formula meant to promote the quality and quantity of milk. This is 100% safe and effective. It can be used as a tonic for physically weak nursing mothers. Breast milk production may increase within a couple of days. However, in order to ensure the best results, the mother should continue taking this preparation for at least 15 days.'},
    {'section': 'Therapeutic Class', 'text': 'Herbal and Nutraceuticals'}],
   'en': [{'section': 'Indications',
     'text': 'This is a natural formula meant to promote the quality and quantity of milk. This is 100% safe and effective. It can be used as a tonic for physically weak nursing mothers. Breast milk production may increase within a couple of days. However, in order to ensure the best results, the mother

In [33]:
import json 
with open('all_herbel_generics.json', 'w', encoding='utf-8') as f:
    json.dump(all_herbal_generics, f, indent=4, ensure_ascii=False)